# WEBSCRAPING CAVEATS

The code contained in this brief tutorial is intended for illustration purposes only. It was not implemented with code design or optimization in mind. It merely shows how to scrape data from various sources using Beautiful Soup.

In [ ]:
from urllib2 import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import unicodedata

# DATA SOURCE #1 - Sports Stats

The website [sports-reference.com](http://www.sports-reference.com/) includes statistics for the MLB, NBA, NFL, NHL, CFB, CBB, and the Olympics. As an example, this tutorial will show how to scrape Chicago Blackhawk player statistics for the 2015-2016 season located [here](http://www.hockey-reference.com/teams/CHI/2016.html) and convert it to a pandas dataframe. 

### GET DATA

In [ ]:
# GET DATA
url = "http://www.hockey-reference.com/teams/CHI/2016.html"
html = urlopen(url)

# PRE-PROCESS
soup = BeautifulSoup(html, "lxml")
table = soup.findAll("table", class_="sortable stats_table")
scoring = table[2]  # subset to just scoring table
column_headers = [th.getText() for th in 
                  scoring.findAll('tr')[1].findAll('th')][1:]  # drop 'Rk'
data_rows = scoring.findAll('tr')[2:]  # skip the first 2 header rows
player_data = [[td.getText() for td in data_rows[i].findAll('td')]
               for i in range(len(data_rows))]

# CREATE DF
df = pd.DataFrame(player_data, columns=column_headers)
df.head()

# DATA SOURCE #2 - Beer

The website [http://hbd.org/ensmingr/](http://hbd.org/ensmingr/) gives the percent alcohol, number of calories, specific gravity before (OG) and after (FG) fermentation, and apparent attenuation for ~1000 commercial beers from ~100 breweries, as of 1999. As an example, this tutorial will show how to scrape data for Grant's Brewery located [here](http://hbd.org/ensmingr/grants.html) and convert it to a pandas dataframe.

In [ ]:
# GET DATA
url2 = "http://hbd.org/ensmingr/grants.html"
html2 = urlopen(url2)

# PRE-PROCESS
soup2 = BeautifulSoup(html2, 'lxml')
column_headers = ['Company and Product', 'Location', 'Percent_Alcohol', 'Calories_Per_12', 'OG', 'FG', 'AA', 'Notes']
data_rows2 = soup2.findAll('tr')[2:]  # skip the first 2 header rows
beer_data = [[td.getText() for td in data_rows2[i].findAll('td')]
             for i in range(len(data_rows2))]
beer_data = [[unicodedata.normalize('NFKD', beer_data[i][j]).encode('ascii','ignore') for j in range(8)] for i in range(len(beer_data))]

# CREATE DF
df2 = pd.DataFrame(beer_data, columns=column_headers).replace({'\r\n': ''}, regex=True)
df2.head()

### WITH PANDAS (much easier)

In [ ]:
grants = pd.read_html(url2, header=0)[0]
grants.head()

# DATA SOURCE #3 - Wine

The website [http://www.wineinstitute.org/resources/pressroom/07082016](http://www.wineinstitute.org/resources/pressroom/07082016) gives wine sales in the U.S. and California.  As an example, this tutorial will show how to scrape wine sales data (in millions of 9-liter cases - from California, other states, and foreign producers entering U.S. distribution) and convert it to a pandas dataframe.. 

In [ ]:
# GET DATA
url3 = "http://www.wineinstitute.org/resources/pressroom/07082016"
html3 = urlopen(url3)

# PRE-PROCESS
soup3 = BeautifulSoup(html3, "lxml")
table3 = soup3.findAll("table")
sales = table3[1]  # subset to just wine sales table
column_headers3 = ['Year', 'Table', 'Dessert', 'Sparkling', 'Total_Wine', 'Total_Retail_Value']
data_rows3 = sales.findAll('tr')[2:]  # skip the first 2 header rows
wine_data = [[td.getText() for td in data_rows3[i].findAll('td')]
             for i in range(len(data_rows3))]

# CREATE DF
df3 = pd.DataFrame(wine_data, columns=column_headers3)
df3.head(10)